In [ ]:
import cv2
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

# Initialize ROS Node

reference: http://wiki.ros.org/rospy/Overview/Initialization%20and%20Shutdown

rospy.init_node(name, anonymous=False, log_level=rospy.INFO, disable_signals=False)

In [ ]:
# Initialize ROS Node
my_node = rospy.init_node('test_node', anonymous=True)

# Define the message rate
rate = rospy.Rate(10)  # 10 Hz

# Message Publisher & Subscriber
reference: http://wiki.ros.org/ROS/Tutorials/WritingPublisherSubscriber%28python%29

## CV_Bridge
reference: http://wiki.ros.org/cv_bridge

![cv_bridge](http://wiki.ros.org/cv_bridge?action=AttachFile&do=get&target=cvbridge.png)

In [ ]:
# Initialize CvBridge
from cv_bridge import CvBridge
bridge = CvBridge()

In [ ]:
# String Publisher
from std_msgs.msg import String
stringPub = rospy.Publisher('test_node/string', String, queue_size=10)

# Image Publisher
from sensor_msgs.msg import Image
imagePub = rospy.Publisher('test_node/image', Image, queue_size=10)

# Publish Messages

In [ ]:
while not rospy.is_shutdown():
    # publish a string message
    strMsg = f"hello world {rospy.get_time()}"
    stringPub.publish(strMsg)

    # publish an image message
    cv_img = cv2.imread("../demo/recordings/demo_video/color/color_000000.jpg")
    img_msg = bridge.cv2_to_imgmsg(cv_img, encoding="bgr8")

    # Add timestamp to the image message
    img_msg.header.stamp = rospy.Time.now()
    imagePub.publish(img_msg)

    rate.sleep()

# Practice

Define a class `RecordingPublisher` that publishes both color and depth images from a video recording. The class should have the following methods:
- `__init__(self, recording_dir, fps=30)`: Initializes the publisher with the given recording directory and publishing rate.
- ` _init_node(self, node_name)`: Initializes the ROS node with the given name.
- `_get_next_frame(self)`: Retrieves the next frame from the video recording in a loop.
- `run(self)`: Publishes the color and depth images at the specified rate.
- The publisher topics are given as dict `PUB_TOPICS`.
- Use `RVIZ` to visualize the published images.

Refer to the code [image_publisher.py](./image_publisher.py) for the answer.

In [ ]:
from pathlib import Path
import cv2
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

PUB_TOPICS = {
    "colorTopic": "/dummy_camera/color/image_raw",
    "depthTopic": "/dummy_camera/depth/image_raw",
}


def read_rgb_image(image_path):
    return cv2.cvtColor(cv2.imread(str(image_path)), cv2.COLOR_BGR2RGB)


def read_depth_image(image_path):
    return cv2.imread(str(image_path), cv2.IMREAD_ANYDEPTH)


class RecordingPublisher:
    def __init__(self, recording_dir, fps=30) -> None:
        # Initialize the Node
        self._init_node("recording_publisher")

        # Initialize the CvBridge
        self._bridge = CvBridge()

        # Define the ros rate
        self._rate = rospy.Rate(fps)

        # Pre-load the color and depth files
        self._color_files = sorted(Path(recording_dir).glob("color/*.jpg"))
        self._depth_files = sorted(Path(recording_dir).glob("depth/*.png"))
        assert len(self._color_files) == len(self._depth_files), "Color and depth files must match in number"

        # Other initializations
        self._num_frames = len(self._color_files)
        self._frame_id = -1

        # Define publishers
        self._colorPub = rospy.Publisher(PUB_TOPICS["colorTopic"], Image, queue_size=1)
        self._depthPub = rospy.Publisher(PUB_TOPICS["depthTopic"], Image, queue_size=1)

    def _init_node(self, node_name):
        """Method to initialize the ROS node with a given name."""
        rospy.loginfo(f"Node initialized: {node_name}")

    def _get_next_frame(self):
        """Method to get the next frame from the recording."""
        self._frame_id = (self._frame_id + 1) % self._num_frames
        color = cv2.imread(str(self._color_files[self._frame_id]), cv2.IMREAD_COLOR)
        depth = cv2.imread(str(self._depth_files[self._frame_id]), cv2.IMREAD_ANYDEPTH)
        return color, depth

    def run(self):
        """Method to start publishing the recording frames."""
        rospy.loginfo("Start publishing recording frames...")

        # Main loop to publish frames
        while not rospy.is_shutdown():
            try:
                # Get the next color and depth frames

                # Convert OpenCV images to ROS messages

                # Add timestamp to the messages


                # Publish the messages


            except CvBridgeError as e:
                rospy.logerr(e)

            self._rate.sleep()

In [ ]:
# Run the RecordingPublisher if this script is executed directly
recording_dir = "../demo/recordings/demo_video"
camera_publisher = RecordingPublisher(recording_dir)
camera_publisher.run()